<a href="https://colab.research.google.com/github/seyed-mohammadreza-mousavi/Retinal-Vessel-Segmentation_A-Computer-Vision-Technique/blob/main/Welcome_To_Colaboratory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
!pip install mtcnn

In [15]:
!mkdir images

mkdir: cannot create directory ‘images’: File exists


In [16]:
!wget https://github.com/Alireza-Akhavan/deep-face-recognition/raw/master/images/alireza.jpg -P images
!wget https://github.com/Alireza-Akhavan/deep-face-recognition/raw/master/images/ali.jpg -P images
!wget https://github.com/Alireza-Akhavan/deep-face-recognition/raw/master/images/mohsen.jpg -P images
!wget https://github.com/Alireza-Akhavan/deep-face-recognition/raw/master/images/muhammad.jpg -P images
!wget https://github.com/Alireza-Akhavan/deep-face-recognition/raw/master/images/1.jpg -P images
!wget https://github.com/Alireza-Akhavan/deep-face-recognition/raw/master/images/srttu-class.png -P images
!wget https://github.com/Alireza-Akhavan/deep-face-recognition/raw/master/images/m_wrapped.png -P images

--2023-10-20 10:35:28--  https://github.com/Alireza-Akhavan/deep-face-recognition/raw/master/images/alireza.jpg
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/Alireza-Akhavan/deep-face-recognition/master/images/alireza.jpg [following]
--2023-10-20 10:35:28--  https://raw.githubusercontent.com/Alireza-Akhavan/deep-face-recognition/master/images/alireza.jpg
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 39450 (39K) [image/jpeg]
Saving to: ‘images/alireza.jpg.1’

alireza.jpg.1       100%[===================>]  38.53K  --.-KB/s    in 0.006s  

2023-10-20 10:35:28 (6.20 MB/s) - ‘images/alireza.jpg.

In [17]:
!wget https://raw.githubusercontent.com/Alireza-Akhavan/deep-face-recognition/master/ArcFace.py

--2023-10-20 10:35:32--  https://raw.githubusercontent.com/Alireza-Akhavan/deep-face-recognition/master/ArcFace.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4043 (3.9K) [text/plain]
Saving to: ‘ArcFace.py.1’

ArcFace.py.1        100%[===================>]   3.95K  --.-KB/s    in 0s      

2023-10-20 10:35:32 (15.6 MB/s) - ‘ArcFace.py.1’ saved [4043/4043]



In [18]:
import ArcFace
from mtcnn.mtcnn import MTCNN
import matplotlib.pyplot as plt
import cv2
from tensorflow.keras.preprocessing import image
import numpy as np
%matplotlib inline

In [19]:
!pip install git+https://github.com/yaledhlab/vggface.git

  Cloning https://github.com/yaledhlab/vggface.git to /tmp/pip-req-build-_z_s3zx3
  Running command git clone --filter=blob:none --quiet https://github.com/yaledhlab/vggface.git /tmp/pip-req-build-_z_s3zx3
  Resolved https://github.com/yaledhlab/vggface.git to commit b76539b7588bca69b0030ad7e4f985f877dc7c0a
  Preparing metadata (setup.py) ... done


In [20]:
!pip install keras_applications

In [21]:
!pip install keras_vggface

In [22]:
# Example of face detection with a vggface2 model
from numpy import expand_dims
from matplotlib import pyplot
from PIL import Image
from numpy import asarray
from mtcnn.mtcnn import MTCNN
from keras_vggface.vggface import VGGFace
from keras_vggface.utils import preprocess_input
from keras_vggface.utils import decode_predictions

In [ ]:
!wget https://github.com/Alireza-Akhavan/deep-face-recognition/raw/master/images/ajb.jpg -P images

In [ ]:
from tensorflow.python.keras import backend
from tensorflow.python.keras.engine import training
from tensorflow.python.keras.utils import data_utils
from tensorflow.python.keras.utils import layer_utils
from tensorflow.python.lib.io import file_io
import tensorflow
from tensorflow import keras

import os
from pathlib import Path
import gdown

def loadModel():
	base_model = ResNet34()
	inputs = base_model.inputs[0]
	arcface_model = base_model.outputs[0]
	arcface_model = keras.layers.BatchNormalization(momentum=0.9, epsilon=2e-5)(arcface_model)
	arcface_model = keras.layers.Dropout(0.4)(arcface_model)
	arcface_model = keras.layers.Flatten()(arcface_model)
	arcface_model = keras.layers.Dense(512, activation=None, use_bias=True, kernel_initializer="glorot_normal")(arcface_model)
	embedding = keras.layers.BatchNormalization(momentum=0.9, epsilon=2e-5, name="embedding", scale=True)(arcface_model)
	model = keras.models.Model(inputs, embedding, name=base_model.name)

	#---------------------------------------
	#check the availability of pre-trained weights

	home = str(Path.home())

	url = "https://drive.google.com/uc?id=1LVB3CdVejpmGHM28BpqqkbZP5hDEcdZY"
	file_name = "arcface_weights.h5"
	output = home+'/.deepface/weights/'+file_name
	Path(home+'/.deepface/weights/').mkdir(parents=True, exist_ok=True)
	if os.path.isfile(output) != True:

		print(file_name," will be downloaded to ",output)
		gdown.download(url, output, quiet=False)

	#---------------------------------------

	try:
		model.load_weights(output)
	except:
		print("pre-trained weights could not be loaded.")
		print("You might try to download it from the url ", url," and copy to ",output," manually")

	return model

def ResNet34():

	img_input = tensorflow.keras.layers.Input(shape=(112, 112, 3))

	x = tensorflow.keras.layers.ZeroPadding2D(padding=1, name='conv1_pad')(img_input)
	x = tensorflow.keras.layers.Conv2D(64, 3, strides=1, use_bias=False, kernel_initializer='glorot_normal', name='conv1_conv')(x)
	x = tensorflow.keras.layers.BatchNormalization(axis=3, epsilon=2e-5, momentum=0.9, name='conv1_bn')(x)
	x = tensorflow.keras.layers.PReLU(shared_axes=[1, 2], name='conv1_prelu')(x)
	x = stack_fn(x)

	model = training.Model(img_input, x, name='ResNet34')

	return model

def block1(x, filters, kernel_size=3, stride=1, conv_shortcut=True, name=None):
	bn_axis = 3

	if conv_shortcut:
		shortcut = tensorflow.keras.layers.Conv2D(filters, 1, strides=stride, use_bias=False, kernel_initializer='glorot_normal', name=name + '_0_conv')(x)
		shortcut = tensorflow.keras.layers.BatchNormalization(axis=bn_axis, epsilon=2e-5, momentum=0.9, name=name + '_0_bn')(shortcut)
	else:
		shortcut = x

	x = tensorflow.keras.layers.BatchNormalization(axis=bn_axis, epsilon=2e-5, momentum=0.9, name=name + '_1_bn')(x)
	x = tensorflow.keras.layers.ZeroPadding2D(padding=1, name=name + '_1_pad')(x)
	x = tensorflow.keras.layers.Conv2D(filters, 3, strides=1, kernel_initializer='glorot_normal', use_bias=False, name=name + '_1_conv')(x)
	x = tensorflow.keras.layers.BatchNormalization(axis=bn_axis, epsilon=2e-5, momentum=0.9, name=name + '_2_bn')(x)
	x = tensorflow.keras.layers.PReLU(shared_axes=[1, 2], name=name + '_1_prelu')(x)

	x = tensorflow.keras.layers.ZeroPadding2D(padding=1, name=name + '_2_pad')(x)
	x = tensorflow.keras.layers.Conv2D(filters, kernel_size, strides=stride, kernel_initializer='glorot_normal', use_bias=False, name=name + '_2_conv')(x)
	x = tensorflow.keras.layers.BatchNormalization(axis=bn_axis, epsilon=2e-5, momentum=0.9, name=name + '_3_bn')(x)

	x = tensorflow.keras.layers.Add(name=name + '_add')([shortcut, x])
	return x

def stack1(x, filters, blocks, stride1=2, name=None):
	x = block1(x, filters, stride=stride1, name=name + '_block1')
	for i in range(2, blocks + 1):
		x = block1(x, filters, conv_shortcut=False, name=name + '_block' + str(i))
	return x

def stack_fn(x):
	x = stack1(x, 64, 3, name='conv2')
	x = stack1(x, 128, 4, name='conv3')
	x = stack1(x, 256, 6, name='conv4')
	return stack1(x, 512, 3, name='conv5')

In [ ]:
model=loadModel()

In [ ]:
model = VGGFace()

In [12]:
model = ArcFace.loadModel()

arcface_weights.h5  will be downloaded to  /root/.deepface/weights/arcface_weights.h5
Access denied with the following error:
pre-trained weights could not be loaded.
You might try to download it from the url  https://drive.google.com/uc?id=1LVB3CdVejpmGHM28BpqqkbZP5hDEcdZY  and copy to  /root/.deepface/weights/arcface_weights.h5  manually



 	Cannot retrieve the public link of the file. You may need to change
	the permission to 'Anyone with the link', or have had many accesses. 

You may still be able to access the file from the browser:

	 https://drive.google.com/uc?id=1LVB3CdVejpmGHM28BpqqkbZP5hDEcdZY 



In [13]:
face_detector = MTCNN()

In [ ]:
# Change the image path with yours.
img = image.load_img('./images/ajb.jpg', target_size=(224, 224))
img

In [85]:
def detect_face(img):
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) #mtcnn expects RGB but OpenCV read BGR
    detections = face_detector.detect_faces(img_rgb)
    detection = detections[0]
    x, y, w, h = detection["box"]
    detected_face = img[int(y):int(y+h), int(x):int(x+w)]
    return detected_face

In [91]:
def preprocess_face(img, target_size=(112,112)):
    img = cv2.imread(img)
    img = detect_face(img)
    img = cv2.resize(img, target_size)
    img_pixels = image.img_to_array(img)
    img_pixels = np.expand_dims(img_pixels, axis = 0)
    img_pixels /= 255 #normalize input in [0, 1]
    return img_pixels

In [92]:
def img_to_encoding(path):
    img = preprocess_face(path)
    return model.predict(img)[0]

In [93]:
database = {}

database["alireza"] = img_to_encoding("./images/alireza.jpg")
database["ali"] = img_to_encoding("./images/ali.jpg")
database["mohsen"] = img_to_encoding("./images/mohsen.jpg")
database["muhammad"] = img_to_encoding("./images/muhammad.jpg")

1/1 [==============================] - 0s 201ms/step


In [22]:
def EuclideanDistance(source_representation, test_representation):
    euclidean_distance = source_representation - test_representation
    euclidean_distance = np.sum(np.multiply(euclidean_distance, euclidean_distance))
    euclidean_distance = np.sqrt(euclidean_distance)
    return euclidean_distance

In [23]:
verification_threshhold = 4.4

In [24]:
def verify(image_path, identity, database):
    # Step 1: Compute the encoding for the image. Use img_to_encoding()
    encoding = img_to_encoding(image_path)

    # Step 2: Compute distance with identity's image
    dist = EuclideanDistance(encoding, database[identity])

    # Step 3: Open the door if dist < verification_threshhold, else don't open
    if dist < verification_threshhold:
        print("It's " + str(identity) + ", welcome!")
    else:
        print("It's not " + str(identity) + ", please go away")

    return dist

In [ ]:
!pip install deepface

In [24]:
from deepface import DeepFace

In [51]:
models = ["VGG-Face", "Facenet", "Facenet512", "OpenFace", "DeepFace", "DeepID", "ArcFace", "Dlib", "SFace",]

In [54]:
result = DeepFace.verify(img1_path = "images/1.jpg", img2_path = "images/alireza.jpg", model_name = models[0])

In [98]:
dfs = DeepFace.find(img_path = "images/mohsen.jpg", db_path = "images/")

There are  13  representations found in  representations_vgg_face.pkl
find function lasts  0.9238824844360352  seconds


In [102]:
embedding_objs = DeepFace.represent(img_path = "images/mohsen.jpg")

In [103]:
embedding = embedding_objs[0]["embedding"]

In [ ]:
embedding

In [99]:
dfs

[               identity  source_x  source_y  source_w  source_h  \
 0  images//mohsen.jpg.1       201       119       367       367   
 1    images//mohsen.jpg       201       119       367       367   
 
    VGG-Face_cosine  
 0     1.110223e-15  
 1     1.110223e-15  ]

In [94]:
result

{'verified': True,
 'distance': 0.2715841925972273,
 'threshold': 0.4,
 'model': 'VGG-Face',
 'detector_backend': 'opencv',
 'similarity_metric': 'cosine',
 'facial_areas': {'img1': {'x': 38, 'y': 18, 'w': 178, 'h': 178},
  'img2': {'x': 189, 'y': 141, 'w': 212, 'h': 212}},
 'time': 1.59}

In [74]:
print(result.get("distance"))

0.2715841925972273


In [ ]:
database

In [ ]:
verify("images/1.jpg", "alireza", database)

In [26]:
verify("images/ali.jpg", "alireza", database)

1/1 [==============================] - 1s 537ms/step
It's alireza, welcome!


0.00023055093